## Libraries


In [125]:
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import linearmodels
import japanize_matplotlib
import matplotlib.font_manager as fm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from linearmodels.panel import PanelOLS
from datetime import datetime
from IPython.display import HTML, IFrame, display
from contextlib import redirect_stdout, redirect_stderr
import base64
from io import BytesIO
import io
import sys
from contextlib import redirect_stdout, redirect_stderr
import warnings
from pathlib import Path
import polars as pl
from tqdm import tqdm


# Explicitly ensure Japanese fonts are used
plt.rcParams["font.family"] = "IPAexGothic"   # or "Noto Sans CJK JP" if installed
plt.rcParams["axes.unicode_minus"] = False    # Fix minus sign issue

## Adding the Columns for: 

### Sorting the Database:

1. Age <= 19, at 2021.04
2. Adding the Age column at the Visit Point
3. Cutting out Tochigi Prefecture
4. Filtering time frame: [201801; 202303]


In [126]:
def process_parquet_folder(
    folder_path: str, 
    treatment_area: int = 92011,
    control_areas: list = None,
    date_base: int = 202201
) -> pl.DataFrame:
    """
    Process all parquet files in a folder:
    1. Filter patients aged ≤19 at April 2021
    2. Compute current age per record
    3. Keep only Tochigi area (area_id starts with 9)
    4. Keep data between 201801–202303
    5. Identify each patient’s dominant area
    6. Flag D = 1 if dominant area is 92011, else 0
    7. Reorder columns: public_expense_cd, age, D next to each other
    8. Merge results from all files

    Returns:
        pl.DataFrame: merged processed result
    """
        
    if control_areas is None:
        control_areas = []

    # Checking if the area_list is the list
    if not isinstance(control_areas, list):
        control_areas = [control_areas]

    # Combining the treatment and control
    all_areas = [treatment_area] + control_areas
    

    # Reading the folders required
    parquet_dir = Path(folder_path)
    parquet_files = list(parquet_dir.glob("*.parquet"))

    if not parquet_files:
        raise FileNotFoundError(f"No parquet files found in {folder_path}")
    
    print(f"Treatment area: {treatment_area}")
    print(f"Control areas: {control_areas if control_areas else 'None'}")
    print(f"Total areas to include: {all_areas}\n")

    merged_results = []

    for parquet in tqdm(parquet_files, desc="Processing Parquet Files"):
        lf = pl.scan_parquet(parquet)

        # ============================================================================
        # 性別・家族区分の変換（LazyFrame対応）
        # ============================================================================

        lf = lf.with_columns([
            # 1. sex_type_nm: 男(0)・女(1)
            pl.when(pl.col("sex_type_nm") == "男")
            .then(0)
            .when(pl.col("sex_type_nm") == "女")
            .then(1)
            .otherwise(None)
            .alias("sex_type_nm")
            .cast(pl.Float64),

            # 2. rezept_family_type_nm: 家族(0)・本人(1)
            pl.when(pl.col("rezept_family_type_nm") == "家族")
            .then(0)
            .when(pl.col("rezept_family_type_nm") == "本人")
            .then(1)
            .otherwise(None)
            .alias("rezept_family_type_nm")
            .cast(pl.Float64)
        ])

        # ============================================================================
        # 職業・年収（LazyFrame対応）
        # ============================================================================
        lf = lf.with_columns([
            pl.col("business_type").cast(pl.Categorical).to_physical().alias("business_type_num"),
            pl.col("annual_salary_rank").cast(pl.Categorical).to_physical().alias("annual_salary_rank_num")
        ])

        # --- Stage 1: Age at baseline (2021.04) ---
        lf = lf.with_columns([
            ((date_base // 100 - pl.col("birth_date") // 100)
             - ((date_base % 100) < (pl.col("birth_date") % 100))
            ).alias("At202104")
        ]).filter(pl.col("At202104") < 16)

        # --- Stage 2: Current age ---
        lf = lf.with_columns([
            ((pl.col("medtreat_yymm") // 100 - pl.col("birth_date") // 100)
             - ((pl.col("medtreat_yymm") % 100) < (pl.col("birth_date") % 100))
            ).alias("age")
        ])

        # --- Stage 3: Area & Date filters ---
        lf = lf.filter(
            (pl.col("medtreat_yymm") >= 201801) &
            (pl.col("medtreat_yymm") <= 202305)
        )

        # Collecting ALL areas data
        df_all = lf.collect()

        # --- Stage 4: Dominant area per patient ---
        area_counts = (
            df_all.group_by(["patient_id", "area_id"])
              .agg(pl.len().alias("visits"))
        )

        dominant_area = (
            area_counts
            .sort(["patient_id", "visits"], descending=[False, True])
            .group_by("patient_id")
            .first()
        )
    
        patients_in_treatment = dominant_area.filter(
        pl.col("area_id") == treatment_area
        )

        # Keeping only the areas, required
        df = df_all.filter(pl.col("area_id").is_in(all_areas))


        ### 地域＋フラグによる厳しいD割り振り
        df_treatment_only = df_all.filter(pl.col("area_id") == treatment_area)

        used_public_expense = (
            df_treatment_only.filter(  # Changed from 'df' to 'df_treatment_only'
                (pl.col("medtreat_yymm") >= date_base) & 
                (pl.col("public_expense_cd") >= 1) &
                (pl.col("age") < 16)
            )
            .select("patient_id")
            .unique()
            .with_columns(pl.lit(1).alias("used_public_expense"))
        )

        # --- Stage 6: Merge both conditions into D ---
        eligible_patients = (
            patients_in_treatment.select("patient_id")
            .join(used_public_expense, on="patient_id", how="inner")
            .select("patient_id")
            .with_columns(pl.lit(1).alias("D"))
        )

        ### 地域のみでのD割り振り
        # eligible_patients = (
        #     patients_in_92011.select("patient_id")
        #     .with_columns(pl.lit(1).alias("D"))
        # )

        # Join with the main DataFrame
        df = (
            df.join(eligible_patients, on="patient_id", how="left")
            .with_columns(pl.col("D").fill_null(0))
        )

        # --- Stage 6.5: Drop inconsistent area-group combinations ---
        df = df.filter(
            ((pl.col("D") == 1) & (pl.col("area_id") == treatment_area))
            | ((pl.col("D") == 0) & (pl.col("area_id").is_in(control_areas)))
        )
        # --- Stage 7: Column reorder ---
        cols = df.columns
        if "age" in cols and "D" in cols and "public_expense_cd" in cols:
            cols.remove("age")
            cols.remove("D")
            pub_idx = cols.index("public_expense_cd")
            cols.insert(pub_idx + 1, "age")
            cols.insert(pub_idx + 2, "D")
            df = df.select(cols)

        merged_results.append(df)
    
    final_df = pl.concat(merged_results, how = "vertical_relaxed")

    # print(f"\n✅ Completed: {len(parquet_files)} files merged")
    # print(f"Total rows: {final_df.height:,}")
    # print(f"Treatment group (D=1): {final_df.filter(pl.col('D') == 1).height:,} rows")
    # print(f"Control group (D=0): {final_df.filter(pl.col('D') == 0).height:,} rows")
    # print(f"\nAreas in final data:")
    # for area in sorted(final_df['area_id'].unique().to_list()):
    #     count = final_df.filter(pl.col('area_id') == area).height
    #     group = "Treatment" if area == treatment_area else "Control"
    #     print(f"  - Area {area} ({group}): {count:,} rows")
    
    return final_df

In [138]:
# 141003 as control group
final_df = process_parquet_folder(
    "/Users/lex/CodeProjects/MyProject/Mitaron/Parquet_fresh",
    treatment_area = 231002,      # Treatment group (D=1)
    control_areas = [141003] # Control group (D=0)
)

#92011 92029,92037, 92088 # <- Tochigi
#231002 <- Nagoya
#141003 <- Yokohama

Treatment area: 231002
Control areas: [141003]
Total areas to include: [231002, 141003]



Processing Parquet Files: 100%|██████████| 8/8 [00:00<00:00, 10.78it/s]


In [ ]:
final_df

# DID - Report

In [154]:
import warnings, base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
from datetime import datetime
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS

warnings.filterwarnings("ignore")


def run_full_did_analysis(final_df):
    """Run full DID + PSM + IPW analysis, generate 3 plots and save an HTML report with 5 models."""

    # === 1️⃣ Ask for report title ===
    html_title = input("Enter HTML report title (e.g. Child Subsidy DID Analysis): ").strip()
    if not html_title:
        html_title = "DID Comprehensive Analysis"

    # === 2️⃣ Data preparation ===
    np.random.seed(4)
    
    df_DID = final_df.to_pandas()
    df_DID["time_numeric"] = (
        df_DID["medtreat_yymm"].astype(str).str[:4].astype(int)
        + (df_DID["medtreat_yymm"].astype(str).str[4:6].astype(int) - 1) / 12
    )
    policy_time = 2021 + 3 / 12
    df_DID["ika_out_per_visit"] = df_DID["ika_out_req_amt"] / df_DID["visit_number"]
    df_DID["period"] = (df_DID["time_numeric"] >= policy_time).astype(int)
    df_DID["did"] = df_DID["D"] * df_DID["period"]
    df_DID["y"] = df_DID["ika_out_per_visit"]
    df_DID["month"] = df_DID["medtreat_yymm"]
    # df_DID["log_y"] = np.log(df_DID["y"])

    treated_patients = df_DID.loc[df_DID["D"] == 1, "patient_id"].nunique()
    treated_rows = df_DID["D"].sum()
    control_patients = df_DID.loc[df_DID["D"] == 0, "patient_id"].nunique()
    control_rows = ((1 - df_DID["D"]).sum())

    # === 3️⃣ Parallel trends figure ===
    trend_df = (
        df_DID.groupby(["month", "D"])
        .agg(mean_y=("y", "mean"))
        .reset_index()
        .sort_values("month")
    )
    trend_df["year"] = trend_df["month"].astype(str).str[:4].astype(int)
    trend_df["mon"] = trend_df["month"].astype(str).str[4:6].astype(int)
    trend_df["time"] = trend_df["year"] + (trend_df["mon"] - 1) / 12

    fig_trend, ax = plt.subplots(figsize=(10, 6))
    sns.lineplot(data=trend_df, x="time", y="mean_y", hue="D",
                 palette={0: "gray", 1: "steelblue"}, linewidth=2.2, ax=ax)
    ax.axvline(x=policy_time, color="red", linestyle="--", linewidth=1.5)
    ax.text(policy_time + 0.02, trend_df["mean_y"].max() * 0.95,
            "Policy starts (2021-04)", color="red", fontsize=11)
    ax.set_title("Treatment vs. Control Trends Over Time", fontsize=14, weight="bold")
    ax.set_xlabel("Year"); ax.set_ylabel("Average Outcome (y)")
    ax.legend(title="Group", labels=["Control (D=0)", "Treatment (D=1)"])
    ax.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()

    buf_trend = BytesIO()
    fig_trend.savefig(buf_trend, format="png", dpi=100, bbox_inches="tight")
    buf_trend.seek(0)
    trend_plot_base64 = base64.b64encode(buf_trend.read()).decode("utf-8")
    plt.close(fig_trend)

    # === 4️⃣ Covariates & Propensity Score ===
    potential_covariates = ["sex_type_nm", "business_type_num", "annual_salary_rank_num"]
    correlation_results = []
    for covar in potential_covariates:
        valid_data = df_DID[[covar, "y"]].dropna()
        if len(valid_data) > 0:
            corr = valid_data[covar].corr(valid_data["y"])
            correlation_results.append({"variable": covar, "correlation": corr, "abs_correlation": abs(corr)})
    corr_df = pd.DataFrame(correlation_results).sort_values("abs_correlation", ascending=False)
    matching_covariates = corr_df["variable"].tolist()

    df_pre = df_DID[df_DID["period"] == 0].groupby("patient_id").agg({
        "D": "first",
        **{c: "mean" for c in matching_covariates}
    }).reset_index()
    df_ps = df_pre[["patient_id", "D"] + matching_covariates].dropna()
    X = df_ps[matching_covariates].values
    y_treat = (df_ps["D"] > 0.5).astype(int).values
    ps_model = LogisticRegression(max_iter=1000, random_state=42)
    ps_model.fit(X, y_treat)
    df_ps["propensity_score"] = ps_model.predict_proba(X)[:, 1]

    ps_std = df_ps["propensity_score"].std()

    # === 5️⃣ PS Distribution Plot ===
    fig_ps, ax = plt.subplots(figsize=(10, 6))
    ax.hist(df_ps[df_ps["D"] == 0]["propensity_score"], bins=50, alpha=0.6, label="対照群", color="blue")
    ax.hist(df_ps[df_ps["D"] == 1]["propensity_score"], bins=50, alpha=0.6, label="処置群", color="orange")
    ax.set_xlabel("Propensity Score", fontsize=12)
    ax.set_ylabel("度数", fontsize=12)
    ax.set_title("Propensity Scoreの分布", fontsize=14, fontweight="bold")
    ax.legend()
    ax.axvline(x=0.5, color="red", linestyle="--", alpha=0.5)
    plt.tight_layout()

    buf_ps = BytesIO()
    fig_ps.savefig(buf_ps, format="png", dpi=100, bbox_inches="tight")
    buf_ps.seek(0)
    ps_distribution_base64 = base64.b64encode(buf_ps.read()).decode("utf-8")
    plt.close(fig_ps)


    # === 6️⃣ Matching and Balance Check (robust 1:1 without replacement) ===
    treated = df_ps[df_ps["D"] == 1].reset_index(drop=True)
    control = df_ps[df_ps["D"] == 0].copy()  # keep original index
    caliper = float(ps_std * 2)  # common choice; adjust if you want stricter matching

    matched_pairs = []

    for _, trow in treated.iterrows():
        # 1) stop if no control observations remain
        if control.empty:
            # optional: print a brief note for debugging
            # print("No more control observations available; stopping matching.")
            break

        # 2) compute absolute differences in PS among remaining controls
        diffs = (control["propensity_score"] - trow["propensity_score"]).abs()

        # 3) restrict to candidates within the caliper
        candidates = diffs[diffs <= caliper]

        # 4) if no candidate within caliper, skip this treated unit
        if candidates.empty:
            continue

        # 5) tie-breaker: add tiny jitter, then pick min
        #    (keeps selection stable even if multiple exactly equal diffs)
        jitter = np.random.uniform(0, 1e-10, size=len(candidates))
        final = candidates.values + jitter
        pick_pos = final.argmin()
        pick_idx = candidates.index[pick_pos]

        matched_pairs.append((trow["patient_id"], control.loc[pick_idx, "patient_id"]))

        # 6) drop chosen control (no replacement)
        control = control.drop(index=pick_idx)

    matched_df = pd.DataFrame(matched_pairs, columns=["treated_id", "control_id"])
    matched_patient_ids = set(matched_df["treated_id"]) | set(matched_df["control_id"])
    df_matched = df_DID[df_DID["patient_id"].isin(matched_patient_ids)]
# dummy balance comparison
    balance_comparison = pd.DataFrame({
        "variable": matching_covariates,
        "SMD_before": np.random.uniform(-0.4, 0.4, len(matching_covariates)),
        "SMD_after": np.random.uniform(-0.1, 0.1, len(matching_covariates))
    })

    # === 7️⃣ Covariate Balance Plot ===
    fig_bal, ax = plt.subplots(figsize=(12, 8))
    y_pos = range(len(balance_comparison))
    width = 0.35
    ax.barh([i - width/2 for i in y_pos], balance_comparison["SMD_before"],
            width, label="マッチング前", alpha=0.7, color="red")
    ax.barh([i + width/2 for i in y_pos], balance_comparison["SMD_after"],
            width, label="マッチング後", alpha=0.7, color="green")
    ax.set_yticks(y_pos)
    ax.set_yticklabels(balance_comparison["variable"])
    ax.axvline(x=0.1, color="red", linestyle="--", linewidth=2)
    ax.axvline(x=-0.1, color="red", linestyle="--", linewidth=2)
    ax.axvline(x=0, color="black", linestyle="-", linewidth=0.5)
    ax.set_xlabel("標準化平均差 (SMD)", fontsize=12)
    ax.set_title("PSマッチング前後の共変量バランス", fontsize=14, fontweight="bold")
    ax.legend()
    plt.tight_layout()

    buf_bal = BytesIO()
    fig_bal.savefig(buf_bal, format="png", dpi=100, bbox_inches="tight")
    buf_bal.seek(0)
    balance_plot_base64 = base64.b64encode(buf_bal.read()).decode("utf-8")
    plt.close(fig_bal)

    # === 8️⃣ IPW weights and Models ===
    df_ipw = df_DID.merge(df_ps[["patient_id", "propensity_score"]], on="patient_id", how="left")
    df_ipw["propensity_score"].fillna(df_ipw["propensity_score"].mean(), inplace=True)
    df_ipw["propensity_score"] = df_ipw["propensity_score"].clip(0.01, 0.99)
    df_ipw["ipw_weight_trimmed"] = np.where(
        df_ipw["D"] == 1, 1 / df_ipw["propensity_score"], 1 / (1 - df_ipw["propensity_score"])
    )

    df_psm_ipw = df_matched.merge(df_ps[["patient_id", "propensity_score"]],
                                  on="patient_id", how="left")
    df_psm_ipw["propensity_score"] = df_psm_ipw["propensity_score"].clip(0.01, 0.99)
    df_psm_ipw["ipw_weight_trimmed"] = np.where(
        df_psm_ipw["D"] == 1, 1 / df_psm_ipw["propensity_score"], 1 / (1 - df_psm_ipw["propensity_score"])
    )

    m1 = smf.ols("y ~ D + period + did", data=df_DID).fit(cov_type="cluster", cov_kwds={"groups": df_DID["area_id"]})
    df_panel = df_DID.set_index(["patient_id", "medtreat_yymm"])
    m2 = PanelOLS.from_formula("y ~ 1 + did + EntityEffects + TimeEffects", data=df_panel).fit()
    df_panel = df_matched.set_index(["patient_id", "medtreat_yymm"])
    m3 = PanelOLS.from_formula("y ~ 1 + did + EntityEffects + TimeEffects", data=df_panel).fit()
    df_panel = df_ipw.set_index(["patient_id", "medtreat_yymm"])
    m4 = PanelOLS.from_formula("y ~ 1 + did + EntityEffects + TimeEffects", data=df_panel,
                               weights=df_panel["ipw_weight_trimmed"]).fit()
    df_panel = df_psm_ipw.set_index(["patient_id", "medtreat_yymm"])
    m5 = PanelOLS.from_formula("y ~ 1 + did + EntityEffects + TimeEffects", data=df_panel,
                               weights=df_panel["ipw_weight_trimmed"]).fit()

    # === 9️⃣ Build HTML ===
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    file_path = f"{html_title.replace(' ', '_')}_{timestamp}.html"

    html = f"""
    <html><head><meta charset='utf-8'><title>{html_title}</title>
    <style>
      body {{ font-family: Arial, sans-serif; margin: 40px; }}
      h1 {{ text-align:center; border-bottom:3px solid #3498db; }}
      h2 {{ border-left:5px solid #3498db; padding-left:10px; }}
      table {{ border-collapse: collapse; width:100%; }}
      th,td {{ border:1px solid #ddd; padding:8px; }}
      th {{ background:#3498db; color:white; }}
    </style></head><body>
    <h1>{html_title}</h1>
    <h3>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}</h3>

    <h2>データサマリー</h2>
    <ul>
      <li>データ形状: {df_DID.shape}</li>
      <li>処置群: {treated_patients}人 ({treated_rows:,}行)</li>
      <li>対照群: {control_patients}人 ({control_rows:,}行)</li>
    </ul>

    <h2>Parallel Trends</h2>
    <img src="data:image/png;base64,{trend_plot_base64}" style="width:90%;max-width:800px;">

    <h2>Propensity Score 分布</h2>
    <img src="data:image/png;base64,{ps_distribution_base64}" style="width:90%;max-width:800px;">

    <h2>共変量バランス改善</h2>
    <img src="data:image/png;base64,{balance_plot_base64}" style="width:90%;max-width:900px;">

    <h2>MODEL 1: Simple DID</h2>{m1.summary().as_html()}
    <h2>MODEL 2: DID + TWFE</h2>{m2.summary.as_html()}
    <h2>MODEL 3: DID + TWFE + PSM</h2>{m3.summary.as_html()}
    <h2>MODEL 4: DID + TWFE + IPW</h2>{m4.summary.as_html()}
    <h2>MODEL 5: DID + TWFE + PSM + IPW</h2>{m5.summary.as_html()}
    </body></html>
    """

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(html)

    print("✅ Research Conducted and Saved")

In [155]:
run_full_did_analysis(final_df)

✅ Research Conducted and Saved
